# Services

## Introducing services

A Kubernetes Service is a resource you create to make a single, constant point of entry to a group of pods providing the same service. Each service has an IP address and port that never change while the service exists. Clients can open connections to that IP and port, and those connections are then routed to one of the pods backing that service. This way, clients of a service don’t need to know the location of individual pods providing the service, allowing those pods to be moved around the cluster at any time.

**Explaining services with an example**

Let’s revisit the example where you have a frontend web server and a backend database server. There may be multiple pods that all act as the frontend, but there may only be a single backend database pod. You need to solve two problems to make the system function:

- External clients need to connect to the frontend pods without caring if there’s only a single web server or hundreds.
- The frontend pods need to connect to the backend database. Because the database runs inside a pod, it may be moved around the cluster over time, causing its IP address to change. You don’t want to reconfigure the frontend pods every time the backend database is moved.

By creating a service for the frontend pods and configuring it to be accessible from outside the cluster, **you expose a single, constant IP address through which external clients** can connect to the pods. Similarly, by also creating a service for the backend pod, you create a stable address for the backend pod. **The service address doesn’t change even if the pod’s IP address changes.** Additionally, by creating the service, you also enable **the frontend pods to easily find the backend service by its name through either environment variables or DNS**. All the components of your system (the two services, the two sets of pods backing those services, and the interdependencies between them) are shown in figure 5.1.

You now understand the basic idea behind services. Now, let’s dig deeper by first seeing how they can be created.

### Creating services

As you’ve seen, a service can be backed by more than one pod. Connections to the service are load-balanced across all the backing pods. But how exactly do you define which pods are part of the service and which aren’t?

You probably remember label selectors and how they’re used in Replication-Controllers and other pod controllers to specify which pods belong to the same set. The same mechanism is used by services in the same way, as you can see in figure 5.2.

In the previous chapter, you created a ReplicationController which then ran three instances of the pod containing the Node.js app. Create the ReplicationController again and verify three pod instances are up and running. After that, you’ll create a Service for those three pods.

    kubectl apply -f ex01-kubia-replicaset.yaml

**Creating a service through kubectl expose**

The easiest way to create a service is through kubectl expose, which you’ve already used in chapter 2 to expose the ReplicationController you created earlier. The expose command created a Service resource with the same pod selector as the one used by the ReplicationController, thereby exposing all its pods through a single IP address and port.

Now, instead of using the expose command, you’ll create a service manually by posting a YAML to the Kubernetes API server.

**Creating a service through a YAML descriptor**

Create a file called kubia-svc.yaml with the following listing’s contents.

```yml
apiVersion: v1
kind: Service
metadata:
  name: kubia
spec:
  ports:
  - port: 80
    targetPort: 8080
  selector:
    app: kubia
```

You’re defining a service called kubia, which will accept connections on port 80 and route each connection to port 8080 of one of the pods matching the app=kubia label selector.

Go ahead and create the service by posting the file using kubectl create.

    kubectl apply -f ex02-kubia-svc.yaml

**Examining your new service**

After posting the YAML, you can list all Service resources in your namespace and see that an internal cluster IP has been assigned to your service:

    kubectl get svc

The list shows that the IP address assigned to the service is 10.111.249.153. Because this is the cluster IP, it’s only accessible from inside the cluster. The primary purpose of services is exposing groups of pods to other pods in the cluster, but you’ll usually also want to expose services externally. You’ll see how to do that later. For now, let’s use your service from inside the cluster and see what it does.

**Testing your service from within the cluster**

You can send requests to your service from within the cluster in a few ways:

- The obvious way is to create a pod that will send the request to the service’s cluster IP and log the response. You can then examine the pod’s log to see what the service’s response was.
- You can ssh into one of the Kubernetes nodes and use the curl command.
- You can execute the curl command inside one of your existing pods through the kubectl exec command.

Let’s go for the last option, so you also learn how to run commands in existing pods.

**Remotely executing commands in running containers**

The kubectl exec command allows you to remotely run arbitrary commands inside an existing container of a pod. This comes in handy when you want to examine the contents, state, and/or environment of a container. List the pods with the kubectl get pods command and choose one as your target for the exec command (in the following example, I’ve chosen the kubia-7nog1 pod as the target). You’ll also need to obtain the cluster IP of your service (using kubectl get svc, for example). When running the following commands yourself, be sure to replace the pod name and the service IP with your own:

     kubectl exec kubia-fkfnb -- curl -s http://10.0.1.160

If you’ve used ssh to execute commands on a remote system before, you’ll recognize that kubectl exec isn’t much different.

> **Why the double dash?**: The double dash (--) in the command signals the end of command options for kubectl. Everything after the double dash is the command that should be executed inside the pod. Using the double dash isn’t necessary if the command has no arguments that start with a dash. But in your case, if you don’t use the double dash there, the -s option would be interpreted as an option for kubectl exec and would result in the following strange and highly misleading error:

Let’s go over what transpired when you ran the command. Figure 5.3 shows the sequence of events. You instructed Kubernetes to execute the curl command inside the container of one of your pods. Curl sent an HTTP request to the service IP, which is backed by three pods. The Kubernetes service proxy intercepted the connection, selected a random pod among the three pods, and forwarded the request to it. Node.js running inside that pod then handled the request and returned an HTTP response containing the pod’s name. Curl then printed the response to the standard output, which was intercepted and printed to its standard output on your local machine by kubectl.

In the previous example, you executed the curl command as a separate process, but inside the pod’s main container. This isn’t much different from the actual main process in the container talking to the service.

**Configuring session affinity on the service**

If you execute the same command a few more times, you should hit a different pod with every invocation, because the service proxy normally forwards each connection to a randomly selected backing pod, even if the connections are coming from the same client.

If, on the other hand, you want all requests made by a certain client to be redirected to the same pod every time, you can set the service’s sessionAffinity property to ClientIP (instead of None, which is the default), as shown in the following listing.

```yml
apiVersion: v1
kind: Service
metadata:
  name: kubia
spec:
  sessionAffinity: ClientIP
  ports:
  - port: 80
    targetPort: 8080
  selector:
    app: kubia
```

    kubectl delete svc kubia
    kubectl apply -f ex03-kubia-svc-client-ip-session-affinity.yaml

    kubectl exec kubia-fkfnb -- curl -s http://10.0.1.160

This makes the service proxy redirect all requests originating from the same client IP to the same pod. As an exercise, you can create an additional service with session affinity set to ClientIP and try sending requests to it.

Kubernetes supports only two types of service session affinity: None and ClientIP. You may be surprised it doesn’t have a cookie-based session affinity option, but you need to understand that Kubernetes services don’t operate at the HTTP level. Services deal with TCP and UDP packets and don’t care about the payload they carry. Because cookies are a construct of the HTTP protocol, services don’t know about them, which explains why session affinity cannot be based on cookies.

**Exposing multiple ports in the same service**

Your service exposes only a single port, but services can also support multiple ports. For example, if your pods listened on two ports—let’s say 8080 for HTTP and 8443 for HTTPS—you could use a single service to forward both port 80 and 443 to the pod’s ports 8080 and 8443. You don’t need to create two different services in such cases. Using a single, multi-port service exposes all the service’s ports through a single cluster IP.

> When creating a service with multiple ports, you must specify a name for each port.

```yml
apiVersion: v1
kind: Service
metadata:
  name: kubia
spec:
  ports:
  - name: http
    port: 80
    targetPort: 8080
  - name: https
    port: 443
    targetPort: 8443
  selector:
    app: kubia
```

> The label selector applies to the service as a whole—it can’t be configured for each port individually. If you want different ports to map to different subsets of pods, you need to create two services.

Because your kubia pods don’t listen on multiple ports, creating a multi-port service and a multi-port pod is left as an exercise to you.

**Using named ports**

In all these examples, you’ve referred to the target port by its number, but you can also give a name to each pod’s port and refer to it by name in the service spec. This makes the service spec slightly clearer, especially if the port numbers aren’t well-known.

For example, suppose your pod defines names for its ports as shown in the following listing.

    kind: Pod
    spec:
      containers:
      - name: kubia
        ports:
        - name: http
          containerPort: 8080
        - name: https
          containerPort: 8443

You can then refer to those ports by name in the service spec, as shown in the following listing.

    apiVersion: v1
    kind: Service
    spec:
      ports:
      - name: http
        port: 80
        targetPort: http
      - name: https
        port: 443
        targetPort: https

But why should you even bother with naming ports? The biggest benefit of doing so is that it enables you to change port numbers later without having to change the service spec. Your pod currently uses port 8080 for http, but what if you later decide you’d like to move that to port 80?

If you’re using named ports, all you need to do is change the port number in the pod spec (while keeping the port’s name unchanged). As you spin up pods with the new ports, client connections will be forwarded to the appropriate port numbers, depending on the pod receiving the connection (port 8080 on old pods and port 80 on the new ones).

### Discovering services

By creating a service, you now have a single and stable IP address and port that you can hit to access your pods. This address will remain unchanged throughout the whole lifetime of the service. Pods behind this service may come and go, their IPs may change, their number can go up or down, but they’ll always be accessible through the service’s single and constant IP address.

But how do the client pods know the IP and port of a service? Do you need to create the service first, then manually look up its IP address and pass the IP to the configuration options of the client pod? Not really. Kubernetes also provides ways for client pods to discover a service’s IP and port.

**Discovering services through environment variables**

When a pod is started, Kubernetes initializes a set of environment variables pointing to each service that exists at that moment. If you create the service before creating the client pods, processes in those pods can get the IP address and port of the service by inspecting their environment variables.

Let’s see what those environment variables look like by examining the environment of one of your running pods. You’ve already learned that you can use the kubectl exec command to run a command in the pod, but because you created the service only after your pods had been created, the environment variables for the service couldn’t have been set yet. You’ll need to address that first.

Before you can see environment variables for your service, you first need to delete all the pods and let the ReplicationController create new ones. You may remember you can delete all pods without specifying their names like this:

    kubectl delete po --all
    kubectl apply -f ex01-kubia-replicaset.yaml

Now you can list the new pods (I’m sure you know how to do that) and pick one as your target for the kubectl exec command. Once you’ve selected your target pod, you can list environment variables by running the env command inside the container, as shown in the following listing.

    kubectl exec kubia-gnsgw env

Two services are defined in your cluster: the kubernetes and the kubia service (you saw this earlier with the kubectl get svc command); consequently, two sets of service-related environment variables are in the list. Among the variables that pertain to the kubia service you created at the beginning of the chapter, you’ll see the `KUBIA_SERVICE_HOST` and the `KUBIA_SERVICE_PORT` environment variables, which hold the IP address and port of the kubia service, respectively.

Turning back to the frontend-backend example we started this chapter with, when you have a frontend pod that requires the use of a backend database server pod, you can expose the backend pod through a service called backend-database and then have the frontend pod look up its IP address and port through the environment variables BACKEND_DATABASE_SERVICE_HOST and BACKEND_DATABASE_SERVICE_PORT.

> NOTE: Dashes in the service name are converted to underscores and all letters are uppercased when the service name is used as the prefix in the environment variable’s name.

Environment variables are one way of looking up the IP and port of a service, but isn’t this usually the domain of DNS? Why doesn’t Kubernetes include a DNS server and allow you to look up service IPs through DNS instead? As it turns out, it does!

**Discovering services through DNS**

Remember in chapter 3 when you listed pods in the kube-system namespace? One of the pods was called kube-dns. The kube-system namespace also includes a corresponding service with the same name.

As the name suggests, the pod runs a DNS server, which all other pods running in the cluster are automatically configured to use (Kubernetes does that by modifying each container’s /etc/resolv.conf file). Any DNS query performed by a process running in a pod will be handled by Kubernetes’ own DNS server, which knows all the services running in your system.

> NOTE: Whether a pod uses the internal DNS server or not is configurable through the dnsPolicy property in each pod’s spec.

Each service gets a DNS entry in the internal DNS server, and client pods that know the name of the service can access it through its fully qualified domain name (FQDN) instead of resorting to environment variables.

**Connecting to the service through its FQDN**

To revisit the frontend-backend example, a frontend pod can connect to the backend-database service by opening a connection to the following FQDN:

    backend-database.default.svc.cluster.local

`backend-database` corresponds to the service name, default stands for the namespace the service is defined in, and `svc.cluster.local` is a configurable cluster domain suffix used in all cluster local service names.



> NOTE: The client must still know the service’s port number. If the service is using a standard port (for example, 80 for HTTP or 5432 for Postgres), that shouldn’t be a problem. If not, the client can get the port number from the environment variable.

Connecting to a service can be even simpler than that. You can omit the `svc.cluster.local` suffix and even the namespace, when the frontend pod is in the same namespace as the database pod. You can thus refer to the service simply as `backend-database`. That’s incredibly simple, right?

Let’s try this. You’ll try to access the kubia service through its FQDN instead of its IP. Again, you’ll need to do that inside an existing pod. You already know how to use kubectl exec to run a single command in a pod’s container, but this time, instead of running the curl command directly, you’ll run the bash shell instead, so you can then run multiple commands in the container. This is similar to what you did in chapter 2 when you entered the container you ran with Docker by using the docker `exec -it bash` command.

**Running a shell in a pod’s container**

You can use the kubectl exec command to run bash (or any other shell) inside a pod’s container. This way you’re free to explore the container as long as you want, without having to perform a kubectl exec for every command you want to run.

> The shell’s binary executable must be available in the container image for this to work.

To use the shell properly, you need to pass the -it option to kubectl exec:

    kubectl exec -it kubia-3inly -- bash

You’re now inside the container. You can use the curl command to access the kubia service in any of the following ways:

    root@kubia-3inly:/# curl http://kubia.default.svc.cluster.local
    You've hit kubia-5asi2

    root@kubia-3inly:/# curl http://kubia.default
    You've hit kubia-3inly

    root@kubia-3inly:/# curl http://kubia
    You've hit kubia-8awf3



You can hit your service by using the service’s name as the hostname in the requested URL. You can omit the namespace and the svc.cluster.local suffix because of how the DNS resolver inside each pod’s container is configured. Look at the `/etc/resolv.conf` file in the container and you’ll understand:

    cat /etc/resolv.conf

**Understanding why you can’t ping a service IP**

One last thing before we move on. You know how to create services now, so you’ll soon create your own. But what if, for whatever reason, you can’t access your service?

You’ll probably try to figure out what’s wrong by entering an existing pod and trying to access the service like you did in the last example. Then, if you still can’t access the service with a simple curl command, maybe you’ll try to ping the service IP to see if it’s up. Let’s try that now:

    ping kubia

Hmm. curl-ing the service works, but pinging it doesn’t. That’s because the service’s cluster IP is a virtual IP, and only has meaning when combined with the service port. We’ll explain what that means and how services work in chapter 11. I wanted to mention that here because it’s the first thing users do when they try to debug a broken service and it catches most of them off guard.

## Connecting to services living outside the cluster

Up to now, we’ve talked about services backed by one or more pods running inside the cluster. But cases exist when you’d like to expose external services through the Kubernetes services feature. Instead of having the service redirect connections to pods in the cluster, you want it to redirect to external IP(s) and port(s).

This allows you to take advantage of both service load balancing and service discovery. Client pods running in the cluster can connect to the external service like they connect to internal services.

###  Introducing service endpoints

Before going into how to do this, let me first shed more light on services. Services don’t link to pods directly. Instead, a resource sits in between—the Endpoints resource. You may have already noticed endpoints if you used the kubectl describe command on your service, as shown in the following listing.

    kubectl describe svc kubia

An Endpoints resource (yes, plural) is a list of IP addresses and ports exposing a service. The Endpoints resource is like any other Kubernetes resource, so you can display its basic info with kubectl get:

    kubectl get endpoints kubia


Although the pod selector is defined in the service spec, it’s not used directly when redirecting incoming connections. Instead, the selector is used to build a list of IPs and ports, which is then stored in the Endpoints resource. When a client connects to a service, the service proxy selects one of those IP and port pairs and redirects the incoming connection to the server listening at that location.

## Exposing services to external clients

Up to now, we’ve only talked about how services can be consumed by pods from inside the cluster. But you’ll also want to expose certain services, such as frontend webservers, to the outside, so external clients can access them, as depicted in figure 5.5.

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/05fig05_alt.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_5-5.png" class="medium-zoom-image">

You have a few ways to make a service accessible externally: 
- Setting the service type to `NodePort` — For a NodePort service, each cluster node opens a port on the node itself (hence the name) and redirects traffic received on that port to the underlying service. The service isn’t accessible only at the internal cluster IP and port, but also through a dedicated port on all nodes.
- Setting the service type to `LoadBalancer`, an extension of the NodePort type — This makes the service accessible through a dedicated load balancer, provisioned from the cloud infrastructure Kubernetes is running on. The load balancer redirects traffic to the node port across all the nodes. Clients connect to the service through the load balancer’s IP.
- Creating an `Ingress` resource, a radically different mechanism for exposing multiple services through a single IP address — It operates at the HTTP level (network layer 7) and can thus offer more features than layer 4 services can. We’ll explain Ingress resources in section 5.4.

###  Using a NodePort service

The first method of exposing a set of pods to external clients is by creating a service and setting its type to NodePort. By creating a NodePort service, you make Kubernetes reserve a port on all its nodes (the same port number is used across all of them) and forward incoming connections to the pods that are part of the service.

This is similar to a regular service (their actual type is ClusterIP), but a NodePort service can be accessed not only through the service’s internal cluster IP, but also through any node’s IP and the reserved node port.

This will make more sense when you try interacting with a NodePort service.

**Creating a NodePort service**

You’ll now create a NodePort service to see how you can use it. The following listing shows the YAML for the service.

```yml
apiVersion: v1
kind: Service
metadata:
  name: kubia-nodeport
spec:
  type: NodePort
  ports:
  - port: 80
    targetPort: 8080
    nodePort: 30123
  selector:
    app: kubia
```

You set the type to NodePort and specify the node port this service should be bound to across all cluster nodes. Specifying the port isn’t mandatory; Kubernetes will choose a random port if you omit it.

    kubectl delete svc kubia
    kubectl apply -f ex04-kubia-svc-nodeport.yaml

> NOTE: When you create the service in GKE, kubectl prints out a warning about having to configure firewall rules. We’ll see how to do that soon.

**Examining your NodePort service**

Let’s see the basic information of your service to learn more about it:

    kubectl get svc kubia-nodeport

The PORT(S) column shows both the internal port of the cluster IP (80) and the node port (30123). The service is accessible at the following addresses:
- 10.111.254.223:80
- <1st node's IP>:30123
- <2nd node's IP>:30123, and so on.

Figure 5.6 shows your service exposed on port 30123 of both of your cluster nodes (this applies if you’re running this on GKE; Minikube only has a single node, but the principle is the same). An incoming connection to one of those ports will be redirected to a randomly selected pod, which may or may not be the one running on the node the connection is being made to.

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/05fig06_alt.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_5-6.png" class="medium-zoom-image">

A connection received on port 30123 of the first node might be forwarded either to the pod running on the first node or to one of the pods running on the second node.

**Changing firewall rules to let external clients access our NodePort service**

As I’ve mentioned previously, before you can access your service through the node port, you need to configure the Google Cloud Platform’s firewalls to allow external connections to your nodes on that port. You’ll do this now:

> AWS: Dodamo security group porte na vse node, ki želimo da so odprti

    gcloud compute firewall-rules list
    gcloud compute firewall-rules create kubia-svc-rule --allow=tcp:30123

You can access your service through port 30123 of one of the node’s IPs. But you need to figure out the IP of a node first. Refer to the sidebar on how to do that.

**Using JSONPath to get the IPs of all your nodes**

You can find the IP in the JSON or YAML descriptors of the nodes. But instead of sifting through the relatively large JSON, you can tell kubectl to print out only the node IP instead of the whole service definition:

    kubectl get nodes -o jsonpath='{.items[*].status.addresses[?(@.type=="ExternalIP")].address}'

You’re telling kubectl to only output the information you want by specifying a JSONPath. You’re probably familiar with XPath and how it’s used with XML. JSONPath is basically XPath for JSON. The JSONPath in the previous example instructs kubectl to do the following:
- Go through all the elements in the items attribute.
- For each element, enter the status attribute.
- Filter elements of the addresses attribute, taking only those that have the type attribute set to ExternalIP.
- Finally, print the address attribute of the filtered elements.

To learn more about how to use JSONPath with kubectl, refer to the documentation at http://kubernetes.io/docs/user-guide/jsonpath.

Once you know the IPs of your nodes, you can try accessing your service through them:

    curl http://130.211.97.55:30123
    You've hit kubia-ym8or
    
    curl http://130.211.99.206:30123
    You've hit kubia-xueq1

As you can see, your pods are now accessible to the whole internet through port 30123 on any of your nodes. It doesn’t matter what node a client sends the request to. But if you only point your clients to the first node, when that node fails, your clients can’t access the service anymore. That’s why it makes sense to put a load balancer in front of the nodes to make sure you’re spreading requests across all healthy nodes and never sending them to a node that’s offline at that moment.

If your Kubernetes cluster supports it (which is mostly true when Kubernetes is deployed on cloud infrastructure), the load balancer can be provisioned automatically by creating a LoadBalancer instead of a NodePort service. We’ll look at this next.

    kubectl delete svc kubia-nodeport
    gcloud compute firewall-rules delete kubia-svc-rule

### Exposing a service through an external load balancer

Kubernetes clusters running on cloud providers usually support the automatic provision of a load balancer from the cloud infrastructure. All you need to do is set the service’s type to LoadBalancer instead of NodePort. The load balancer will have its own unique, publicly accessible IP address and will redirect all connections to your service. You can thus access your service through the load balancer’s IP address.

If Kubernetes is running in an environment that doesn’t support LoadBalancer services, the load balancer will not be provisioned, but the service will still behave like a NodePort service. That’s because a LoadBalancer service is an extension of a NodePort service. You’ll run this example on Google Kubernetes Engine, which supports LoadBalancer services. Minikube doesn’t, at least not as of this writing.

**Creating a LoadBalancer service**

To create a service with a load balancer in front, create the service from the following YAML manifest, as shown in the following listing.

```yml
apiVersion: v1
kind: Service
metadata:
  name: kubia-loadbalancer
spec:
  type: LoadBalancer
  ports:
  - port: 80
    targetPort: 8080
  selector:
    app: kubia
```

The service type is set to LoadBalancer instead of NodePort. You’re not specifying a specific node port, although you could (you’re letting Kubernetes choose one instead).

    kubectl apply -f ex05-kubia-svc-loadbalancer.yaml

**Connecting to the service through the load balancer**

After you create the service, it takes time for the cloud infrastructure to create the load balancer and write its IP address into the Service object. Once it does that, the IP address will be listed as the external IP address of your service:

    kubectl get svc kubia-loadbalancer

In this case, the load balancer is available at IP 130.211.53.173, so you can now access the service at that IP address:

    curl http://130.211.53.173

Success! As you may have noticed, this time you didn’t need to mess with firewalls the way you had to before with the NodePort service.



**Session affinity and web browsers**

Because your service is now exposed externally, you may try accessing it with your web browser. You’ll see something that may strike you as odd—the browser will hit the exact same pod every time. Did the service’s session affinity change in the meantime? With kubectl describe, you can double-check that the service’s session affinity is still set to None, so why don’t different browser requests hit different pods, as is the case when using curl?

Let me explain what’s happening. The browser is using keep-alive connections and sends all its requests through a single connection, whereas curl opens a new connection every time. Services work at the connection level, so when a connection to a service is first opened, a random pod is selected and then all network packets belonging to that connection are all sent to that single pod. Even if session affinity is set to None, users will always hit the same pod (until the connection is closed).

See figure 5.7 to see how HTTP requests are delivered to the pod. External clients (curl in your case) connect to port 80 of the load balancer and get routed to the implicitly assigned node port on one of the nodes. From there, the connection is forwarded to one of the pod instances.

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/05fig07_alt.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_5-7.png" class="medium-zoom-image">

As already mentioned, a LoadBalancer-type service is a NodePort service with an additional infrastructure-provided load balancer. If you use kubectl describe to display additional info about the service, you’ll see that a node port has been selected for the service. If you were to open the firewall for this port, the way you did in the previous section about NodePort services, you could access the service through the node IPs as well.


### Understanding the peculiarities of external connections

You must be aware of several things related to externally originating connections to services.

**Understanding and preventing unnecessary network hops**

When an external client connects to a service through the node port (this also includes cases when it goes through the load balancer first), the randomly chosen pod may or may not be running on the same node that received the connection. An additional network hop is required to reach the pod, but this may not always be desirable.

You can prevent this additional hop by configuring the service to redirect external traffic only to pods running on the node that received the connection. This is done by setting the externalTrafficPolicy field in the service’s spec section:

    spec:
      externalTrafficPolicy: Local
      ...

If a service definition includes this setting and an external connection is opened through the service’s node port, the service proxy will choose a locally running pod. If no local pods exist, the connection will hang (it won’t be forwarded to a random global pod, the way connections are when not using the annotation). You therefore need to ensure the load balancer forwards connections only to nodes that have at least one such pod.

Using this annotation also has other drawbacks. Normally, connections are spread evenly across all the pods, but when using this annotation, that’s no longer the case.

Imagine having two nodes and three pods. Let’s say node A runs one pod and node B runs the other two. If the load balancer spreads connections evenly across the two nodes, the pod on node A will receive 50% of all connections, but the two pods on node B will only receive 25% each, as shown in figure 5.8.

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/05fig08_alt.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_5-8.png" class="medium-zoom-image">

**Being aware of the non-preservation of the client’s IP**

Usually, when clients inside the cluster connect to a service, the pods backing the service can obtain the client’s IP address. But when the connection is received through a node port, the packets’ source IP is changed, because Source Network Address Translation (SNAT) is performed on the packets.

The backing pod can’t see the actual client’s IP, which may be a problem for some applications that need to know the client’s IP. In the case of a web server, for example, this means the access log won’t show the browser’s IP.

The Local external traffic policy described in the previous section affects the preservation of the client’s IP, because there’s no additional hop between the node receiving the connection and the node hosting the target pod (SNAT isn’t performed).

    kubectl delete svc kubia-loadbalancer

##  Exposing services externally through an Ingress resource

You’ve now seen two ways of exposing a service to clients outside the cluster, but another method exists—creating an Ingress resource.

> DEFINITION: Ingress (noun)—The act of going in or entering; the right to enter; a means or place of entering; entryway.

Let me first explain why you need another way to access Kubernetes services from the outside.

**Understanding why Ingresses are needed**

One important reason is that each LoadBalancer service requires its own load balancer with its own public IP address, whereas an Ingress only requires one, even when providing access to dozens of services. When a client sends an HTTP request to the Ingress, the host and path in the request determine which service the request is forwarded to, as shown in figure 5.9.

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/05fig09_alt.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_5-9.png" class="medium-zoom-image">

Ingresses operate at the application layer of the network stack (HTTP) and can provide features such as cookie-based session affinity and the like, which services can’t.

**Understanding that an Ingress controller is required**

Before we go into the features an Ingress object provides, let me emphasize that to make Ingress resources work, an Ingress controller needs to be running in the cluster. Different Kubernetes environments use different implementations of the controller, but several don’t provide a default controller at all.

For example, Google Kubernetes Engine uses Google Cloud Platform’s own HTTP load-balancing features to provide the Ingress functionality.

### Creating an Ingress resource

You’ve confirmed there’s an Ingress controller running in your cluster, so you can now create an Ingress resource. The following listing shows what the YAML manifest for the Ingress looks like.

```yml
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: kubia
spec:
  rules:
  - host: kubia.example.com
    http:
      paths:
      - path: /
        backend:
          serviceName: kubia-nodeport
          servicePort: 80
```

This defines an Ingress with a single rule, which makes sure all HTTP requests received by the Ingress controller, in which the host kubia.example.com is requested, will be sent to the kubia-nodeport service on port 80.

> NOTE: Ingress controllers on cloud providers (in GKE, for example) require the Ingress to point to a NodePort service. But that’s not a requirement of Kubernetes itself.

    kubectl apply -f ex04-kubia-svc-nodeport.yaml
    kubectl apply -f ex06-kubia-ingress.yaml

###  GCloud: Accessing the service through the Ingress

To access your service through http://kubia.example.com, you’ll need to make sure the domain name resolves to the IP of the Ingress controller.

**Obtaining the IP address of the Ingress**

To look up the IP, you need to list Ingresses:

    kubectl get ingresses

> NOTE: When running on cloud providers, the address may take time to appear, because the Ingress controller provisions a load balancer behind the scenes.

The IP is shown in the ADDRESS column.

**Ensuring the host configured in the Ingress points to the Ingress’ IP address**

Once you know the IP, you can then either configure your DNS servers to resolve kubia.example.com to that IP or you can add the following line to `/etc/hosts` (or C:\windows\system32\drivers\etc\hosts on Windows):

    192.168.99.100    kubia.example.com

**Accessing pods through the Ingress**


Everything is now set up, so you can access the service at http://kubia.example.com (using a browser or curl):

    curl http://kubia.example.com

You’ve successfully accessed the service through an Ingress. Let’s take a better look at how that unfolded.

**Understanding how Ingresses work**

Figure 5.10 shows how the client connected to one of the pods through the Ingress controller. The client first performed a DNS lookup of kubia.example.com, and the DNS server (or the local operating system) returned the IP of the Ingress controller. The client then sent an HTTP request to the Ingress controller and **specified kubia.example.com in the Host header**. From that header, the controller determined which service the client is trying to access, looked up the pod IPs through the Endpoints object associated with the service, and forwarded the client’s request to one of the pods.

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/05fig10_alt.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_5-10.png" class="medium-zoom-image">

As you can see, the Ingress controller didn’t forward the request to the service. It only used it to select a pod. Most, if not all, controllers work like this.

### Exposing multiple services through the same Ingress

If you look at the Ingress spec closely, you’ll see that both rules and paths are arrays, so they can contain multiple items. An Ingress can map multiple hosts and paths to multiple services, as you’ll see next. Let’s focus on paths first.

**Mapping different services to different paths of the same host**

You can map multiple paths on the same host to different services, as shown in the following listing.

    ...
      - host: kubia.example.com
        http:
          paths:
          - path: /kubia
            backend:
              serviceName: kubia
              servicePort: 80
          - path: /bar
            backend:
              serviceName: bar
              servicePort: 80

In this case, requests will be sent to two different services, depending on the path in the requested URL. Clients can therefore reach two different services through a single IP address (that of the Ingress controller).

**Mapping different services to different hosts**

Similarly, you can use an Ingress to map to different services based on the host in the HTTP request instead of (only) the path, as shown in the next listing.

    spec:
      rules:
      - host: foo.example.com
        http:
          paths:
          - path: /
            backend:
              serviceName: foo
              servicePort: 80
      - host: bar.example.com
        http:
          paths:
          - path: /
            backend:
              serviceName: bar
              servicePort: 80

Requests received by the controller will be forwarded to either service foo or bar, depending on the Host header in the request (the way virtual hosts are handled in web servers). DNS needs to point both the foo.example.com and the bar.example.com domain names to the Ingress controller’s IP address.

    kubectl delete ingresses kubia
    kubectl delete svc kubia-nodeport

###  AWS: Accessing the service through the Ingress

- [How do I set up the ALB Ingress Controller on an Amazon EC2 node group in Amazon EKS?](https://aws.amazon.com/premiumsupport/knowledge-center/eks-alb-ingress-controller-setup/)
- [ALB Ingress Controller on Amazon EKS](https://docs.aws.amazon.com/eks/latest/userguide/alb-ingress.html)
- https://www.eksworkshop.com/beginner/130_exposing-service/ingress_controller_alb/
- https://github.com/kubernetes-sigs/aws-alb-ingress-controller
- https://kubernetes.cn/docs/concepts/services-networking/ingress-controllers/
- https://docs.aws.amazon.com/eks/latest/userguide/network_reqs.html#vpc-subnet-tagging

Deploy a sample application to verify that the ALB Ingress Controller creates an Application Load Balancer because of the Ingress object.

1.    To deploy a game called 2048 as a sample application, run the following commands:

    
    kubectl apply -f 2048-deployment.yaml
    
    kubectl apply -f 2048-service.yaml
    
    kubectl apply -f 2048-ingress.yaml

To verify that the Ingress resource was created, wait a few minutes, and then run the following command:

    kubectl get ingress/2048-ingress

If your Ingress isn't created after several minutes, run the following command to view the ALB Ingress Controller logs:

    kubectl logs -n kube-system deployment.apps/alb-ingress-controller

    kubectl delete -f 2048_game/

## Troubleshooting services

Services are a crucial Kubernetes concept and the source of frustration for many developers. I’ve seen many developers lose heaps of time figuring out why they can’t connect to their pods through the service IP or FQDN. For this reason, a short look at how to troubleshoot services is in order.

When you’re unable to access your pods through the service, you should start by going through the following list:
- First, make sure you’re connecting to the service’s cluster IP from within the cluster, not from the outside.
- Don’t bother pinging the service IP to figure out if the service is accessible (remember, the service’s cluster IP is a virtual IP and pinging it will never work).
- If you’ve defined a readiness probe, make sure it’s succeeding; otherwise the pod won’t be part of the service.
- To confirm that a pod is part of the service, examine the corresponding Endpoints object with kubectl get endpoints.
- If you’re trying to access the service through its FQDN or a part of it (for example, myservice.mynamespace.svc.cluster.local or myservice.mynamespace) and it doesn’t work, see if you can access it using its cluster IP instead of the FQDN.
- Check whether you’re connecting to the port exposed by the service and not the target port.
- Try connecting to the pod IP directly to confirm your pod is accepting connections on the correct port.
- If you can’t even access your app through the pod’s IP, make sure your app isn’t only binding to localhost.